In [1]:
# Notebook parameters. Values here are for development only and 
# will be overridden when running via snakemake and papermill.
cohort_id = "CD-NU_Gbadolite_gamb_2015_Q3"
# cohort_id = 'ML-2_Kati_colu_2014_Q3'
# cohort_id = 'CI-LG_Agneby-Tiassa_colu_2012'
cohorts_analysis = "20230223"
contigs = ['2RL', '3RL', 'X']
sample_sets = "3.0"
min_cohort_size = 20
max_cohort_size = 50
h12_calibration_contig = '3L'
use_gcs_cache = False
dask_scheduler = "threads"

In [2]:
# Parameters
min_cohort_size = 20
max_cohort_size = 50
sample_sets = ["3.0"]
contigs = ["2RL", "3RL", "X"]
cohorts_analysis = "20230223"
h12_calibration_contig = "3L"
h12_signal_detection_min_delta_aic = 1000
h12_signal_detection_min_stat_max = 0.1
h12_signal_detection_gflanks = [6]
use_gcs_cache = False
dask_scheduler = "single-threaded"
cohort_id = "MZ-I_Morrumbene_gamb_2004_Q2"


In [3]:
from IPython.display import Markdown, HTML
import malariagen_data
import numpy as np
import pandas as pd
from pyprojroot import here
import yaml
import dask
dask.config.set(scheduler=dask_scheduler);
from textwrap import dedent

import geopandas as gpd
import bokeh.layouts as bklay
import bokeh.plotting as bkplt
import bokeh.models as bkmod
import plotly.express as px

from bokeh.io import output_notebook # enables plot interface in J notebook

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

# Mozambique / Morrumbene / gambiae / 2004 / Q2

In [4]:
# Load cohorts to find sample query to select samples for this cohort.

df_cohorts = gpd.read_file(here() / "build" / "final_cohorts.geojson").set_index("cohort_id")
cohort = df_cohorts.loc[cohort_id]
cohort

cohort_size                                                          22
country                                                      Mozambique
admin1_iso                                                         MZ-I
admin1_name                                                   Inhambane
admin2_name                                                  Morrumbene
taxon                                                           gambiae
year                                                               2004
quarter                                                               2
cohort_label              Mozambique / Morrumbene / gambiae / 2004 / Q2
sample_query          cohort_admin2_quarter == 'MZ-I_Morrumbene_gamb...
latitude                                                        -23.716
longitude                                                        35.299
h12_window_size                                                 10000.0
country_alpha2                                                  

In [5]:
# Initialise access to MalariaGEN data.

extra_params = dict()
if use_gcs_cache:
    extra_params["url"] = "simplecache::gs://vo_agam_release"
    extra_params["simplecache"] = dict(cache_storage=(here() / "gcs_cache").as_posix())

ag3 = malariagen_data.Ag3(
    # Pin the version of the cohorts analysis for reproducibility.
    cohorts_analysis=cohorts_analysis,
    results_cache=(here() / "malariagen_data_cache").as_posix(),
    **extra_params,
)
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release/
Data releases available : 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8
Results cache           : /home/sanjaynagi/selection-atlas/malariagen_data_cache
Cohorts analysis        : 20230223
AIM analysis            : 20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 7.13.0
Client location         : unknown
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see 
https://malariagen.github.io/vector-data/ag3/api.html

In [6]:
# Load sample metadata for this cohort.

df_samples = ag3.sample_metadata(sample_query=cohort['sample_query'])
df_samples

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,...,admin1_name,admin1_iso,admin2_name,taxon,cohort_admin1_year,cohort_admin1_month,cohort_admin1_quarter,cohort_admin2_year,cohort_admin2_month,cohort_admin2_quarter
0,BQ0092-C,12/1,Joao Pinto,Mozambique,Furvela,2004,4,-23.716,35.299,F,...,Inhambane,MZ-I,Morrumbene,gambiae,MZ-I_gamb_2004,MZ-I_gamb_2004_04,MZ-I_gamb_2004_Q2,MZ-I_Morrumbene_gamb_2004,MZ-I_Morrumbene_gamb_2004_04,MZ-I_Morrumbene_gamb_2004_Q2
1,BQ0093-C,12/2,Joao Pinto,Mozambique,Furvela,2004,4,-23.716,35.299,F,...,Inhambane,MZ-I,Morrumbene,gambiae,MZ-I_gamb_2004,MZ-I_gamb_2004_04,MZ-I_gamb_2004_Q2,MZ-I_Morrumbene_gamb_2004,MZ-I_Morrumbene_gamb_2004_04,MZ-I_Morrumbene_gamb_2004_Q2
2,BQ0094-C,12/4,Joao Pinto,Mozambique,Furvela,2004,4,-23.716,35.299,F,...,Inhambane,MZ-I,Morrumbene,gambiae,MZ-I_gamb_2004,MZ-I_gamb_2004_04,MZ-I_gamb_2004_Q2,MZ-I_Morrumbene_gamb_2004,MZ-I_Morrumbene_gamb_2004_04,MZ-I_Morrumbene_gamb_2004_Q2
3,BQ0095-C,12/5,Joao Pinto,Mozambique,Furvela,2004,4,-23.716,35.299,F,...,Inhambane,MZ-I,Morrumbene,gambiae,MZ-I_gamb_2004,MZ-I_gamb_2004_04,MZ-I_gamb_2004_Q2,MZ-I_Morrumbene_gamb_2004,MZ-I_Morrumbene_gamb_2004_04,MZ-I_Morrumbene_gamb_2004_Q2
4,BQ0096-C,12/7,Joao Pinto,Mozambique,Furvela,2004,4,-23.716,35.299,F,...,Inhambane,MZ-I,Morrumbene,gambiae,MZ-I_gamb_2004,MZ-I_gamb_2004_04,MZ-I_gamb_2004_Q2,MZ-I_Morrumbene_gamb_2004,MZ-I_Morrumbene_gamb_2004_04,MZ-I_Morrumbene_gamb_2004_Q2
5,BQ0097-C,12/8,Joao Pinto,Mozambique,Furvela,2004,4,-23.716,35.299,F,...,Inhambane,MZ-I,Morrumbene,gambiae,MZ-I_gamb_2004,MZ-I_gamb_2004_04,MZ-I_gamb_2004_Q2,MZ-I_Morrumbene_gamb_2004,MZ-I_Morrumbene_gamb_2004_04,MZ-I_Morrumbene_gamb_2004_Q2
6,BQ0098-C,12/9,Joao Pinto,Mozambique,Furvela,2004,4,-23.716,35.299,F,...,Inhambane,MZ-I,Morrumbene,gambiae,MZ-I_gamb_2004,MZ-I_gamb_2004_04,MZ-I_gamb_2004_Q2,MZ-I_Morrumbene_gamb_2004,MZ-I_Morrumbene_gamb_2004_04,MZ-I_Morrumbene_gamb_2004_Q2
7,BQ0099-C,13/1,Joao Pinto,Mozambique,Furvela,2004,4,-23.716,35.299,F,...,Inhambane,MZ-I,Morrumbene,gambiae,MZ-I_gamb_2004,MZ-I_gamb_2004_04,MZ-I_gamb_2004_Q2,MZ-I_Morrumbene_gamb_2004,MZ-I_Morrumbene_gamb_2004_04,MZ-I_Morrumbene_gamb_2004_Q2
8,BQ0100-C,13/2,Joao Pinto,Mozambique,Furvela,2004,4,-23.716,35.299,F,...,Inhambane,MZ-I,Morrumbene,gambiae,MZ-I_gamb_2004,MZ-I_gamb_2004_04,MZ-I_gamb_2004_Q2,MZ-I_Morrumbene_gamb_2004,MZ-I_Morrumbene_gamb_2004_04,MZ-I_Morrumbene_gamb_2004_Q2
9,BQ0102-C,13/4,Joao Pinto,Mozambique,Furvela,2004,4,-23.716,35.299,F,...,Inhambane,MZ-I,Morrumbene,gambiae,MZ-I_gamb_2004,MZ-I_gamb_2004_04,MZ-I_gamb_2004_Q2,MZ-I_Morrumbene_gamb_2004,MZ-I_Morrumbene_gamb_2004_04,MZ-I_Morrumbene_gamb_2004_Q2


In [7]:
# Determine collection dates.

df_collection_dates = (
    df_samples
    .groupby(["year", "month"])
    .size()
    .reset_index()
    .rename(columns={0: 'count'})
)
df_collection_dates

,year,month,count
0,2004,4,22


In [8]:
# Determine first and last collection months.

min_month = df_collection_dates["month"].min()
max_month = df_collection_dates["month"].max()

if min_month < 0:
    start_month = end_month = None
else:
    start_month = pd.to_datetime(min_month, format="%m").month_name()
    end_month = pd.to_datetime(max_month, format="%m").month_name()
    
start_month, end_month

('April', 'April')

In [9]:
# Determine unique collection locations.

df_locations = df_samples[["location", "longitude", "latitude"]].drop_duplicates()
df_locations

,location,longitude,latitude
0,Furvela,35.299,-23.716


In [10]:
# Extract provenance information about the samples.

df_contributors = df_samples[["release", "sample_set", "contributor"]].drop_duplicates()
df_contributors["study"] = df_contributors.apply(
    lambda v: "Ag1000G" if v["sample_set"].startswith("AG1000G") else "TODO",
    axis="columns"
)
df_contributors["release"] = df_contributors["release"].apply(
    lambda v: f"Ag{v}"
)
df_contributors.rename(columns={
    "contributor": "Contributor",
    "study": "Study",
    "release": "Data release",
    "sample_set": "Sample set",
}, inplace=True)
df_contributors.set_index(["Contributor", "Study", "Data release"], inplace=True)
df_contributors

,,,Sample set
Contributor,Study,Data release,
Joao Pinto,Ag1000G,Ag3.0,AG1000G-MZ


In [11]:
# Build a paragraph with summary information about the samples in this cohort.

n_locations = len(df_locations)

summary_info = f"""This cohort comprises {cohort['cohort_size']:,} samples from the 
*{cohort['taxon']}* taxon, collected from {n_locations} locations within the administrative 
division of {cohort['admin2_name']}, {cohort['admin1_name']}, {cohort['country']}."""

if start_month and start_month == end_month:
    summary_info += f""" Collections were made in {start_month} {cohort['year']}."""
elif start_month:
    summary_info += f""" Collections were made between {start_month} and {end_month} in {cohort['year']}."""
else:
    summary_info += f""" Collections were made in {cohort['year']}."""
    
display(Markdown(summary_info))


This cohort comprises 22 samples from the 
*gambiae* taxon, collected from 1 locations within the administrative 
division of Morrumbene, Inhambane, Mozambique. Collections were made in April 2004.

## Selection scans

In [12]:
# load signals to overlay on H12 plots
import pandas as pd

df_signals_contigs = []
for contig in contigs:

    try:
        df_signals_contig = pd.read_csv(here() / "build/h12-signal-detection/" / f"{cohort_id}_{contig}.csv")
    except pd.errors.EmptyDataError:
        df_signals_contig = pd.DataFrame()
    df_signals_contigs.append(df_signals_contig)  
    
df_signals = pd.concat(df_signals_contigs)

# Add extra columns to help with overlaying signals on plots.
df_signals["bottom"] = 0
df_signals["top"] = 1

df_signals

,bottom,top


In [13]:
# load window sizes 
h12_calibration_dir = "build/h12-calibration"
with open(here() / h12_calibration_dir / f"{cohort_id}.yaml") as h12_calibration_file:
    h12_calibration_params = yaml.safe_load(h12_calibration_file)
h12_window_size = h12_calibration_params["h12_window_size"]
g123_calibration_dir = "build/g123-calibration"
with open(here() / g123_calibration_dir / f"{cohort_id}.yaml") as g123_calibration_file:
    g123_calibration_params = yaml.safe_load(g123_calibration_file)
g123_window_size = g123_calibration_params["g123_window_size"]

if cohort.taxon == 'arabiensis':
    phasing_analysis = 'arab'
else:
    phasing_analysis = 'gamb_colu'

ihs_window_size = 100

def plot_h12_g123_ihs_tracks(
        contig, 
        df_signals,
        sizing_mode='stretch_width', 
        show=False, 
        width=800, 
        track_height=150,
        genes_height=90,
    ):

    sample_query = cohort["sample_query"]
    
    fig1 = ag3.plot_h12_gwss_track(
        contig=contig, 
        window_size=h12_window_size, 
        analysis=phasing_analysis, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        show=show,
        width=width,
        height=track_height,
    )
    fig1.xaxis.visible = False
        
    if not df_signals.empty:
        df = df_signals.query("contig == @contig")
        center_xs = [np.array([row.pcenter, row.pcenter]) for idx, row in df.iterrows()]
        center_ys = [np.array([0, 1]) for idx, row in df.iterrows()]
        source = bkmod.ColumnDataSource(data={
            'cohort': df.cohort_id,
            'chromosome': df.contig,
            'score': df.delta_i.astype(int),
            'peak_start': df.span2_pstart,
            'peak_stop': df.span2_pstop,
            'focus_start': df.focus_pstart,
            'focus_stop': df.focus_pstop,    
            'center_xs': center_xs,
            'center_ys': center_ys,
            'bottom': df.bottom ,
            'top': df.top,
            })
        quad = fig1.quad(bottom='bottom', top='top', left='peak_start', right='peak_stop', 
              source=source, color="gray", alpha=.1, line_width=1)
        quad2 = fig1.quad(bottom='bottom', top='top', left='focus_start', right='focus_stop', 
              source=source, color="red", alpha=.4)
        glyph = bkmod.MultiLine(xs='center_xs', ys='center_ys', line_color='red', line_width=2, line_alpha=0.8)
        fig1.add_glyph(source, glyph)
    
        hover = bkmod.HoverTool(tooltips=[
            ("Cohort", '@cohort'),
            ("Score", '@score'),
            ("Focus", "@focus_start{,} - @focus_stop{,}"),
        ], renderers=[quad])

        fig1.add_tools(hover)
    
    fig2 = ag3.plot_g123_gwss_track(
        contig=contig, 
        window_size=g123_window_size, 
        sites=phasing_analysis,
        site_mask=phasing_analysis, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        width=width,
        height=track_height,
        show=show,
        title="",
        x_range=fig1.x_range,
    )
    fig2.xaxis.visible = False

    fig3 = ag3.plot_ihs_gwss_track(
        contig=contig, 
        window_size=ihs_window_size, 
        analysis=phasing_analysis, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        min_cohort_size=min_cohort_size,
        max_cohort_size=max_cohort_size,
        sizing_mode=sizing_mode,
        width=width,
        height=track_height,
        show=show,
        title="",
        x_range=fig1.x_range,
    )
    fig3.xaxis.visible = False


    fig4 = ag3.plot_genes(
        region=contig, 
        show=show,
        sizing_mode=sizing_mode,
        width=width,
        height=genes_height,
        x_range=fig1.x_range
        )
                        
    fig = bklay.gridplot(
        [fig1, fig2, fig3, fig4],
        ncols=1,
        toolbar_location="above",
        merge_tools=True,
        sizing_mode=sizing_mode,
    )
    return fig 

In [14]:
for contig in contigs:
    
    display(HTML(f"<h3>Chromosome {contig}</h3>"))
    
    fig = plot_h12_g123_ihs_tracks(
        contig=contig,
        df_signals=df_signals,
    );

    bkplt.show(fig)


## Sampling information

In [15]:
from ipyleaflet import Map, Marker, basemaps
from ipywidgets import HTML

center = cohort[['latitude', 'longitude']].to_list()
m = Map(center=center, zoom=9, basemap=basemaps.OpenTopoMap)

df = df_samples[['latitude', 'longitude', 'taxon']].groupby(['latitude', 'longitude', 'taxon']).size().to_frame().rename(columns={0: 'count'}).reset_index()

for coh_id, row in df.iterrows():
    lat, long = row[['latitude', 'longitude']]
    
    if row['taxon'] == 'gambiae':
        color= 'red'
    elif row['taxon'] == 'coluzzii':
        color='cadetblue'
    elif row['taxon'] == 'arabiensis':
        color='lightgreen'
    else: 
        color='gray'

    marker = Marker(location=(lat, long), draggable=False, opacity=0.7, color=color)
    m.add_layer(marker);

    message2 = HTML()
    message2.value = f"n = {row['count']}"
    marker.popup = message2

display(m)

Map(center=[-23.716, 35.299], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [16]:

if min_month >= 0:
    # For this cohort we have month data, so show a breakdown of sample
    # numbers by location and month.
    
    # Construct a pivot table counting samples.
    df_loc_dt = df_samples.pivot_table(
        index=["location", "longitude", "latitude"],
        columns="month",
        values="sample_id",
        aggfunc="count",
        fill_value=0,
    )

    # Tidy up the columns using a multi index.
    df_loc_dt.reset_index(inplace=True)
    cols = pd.MultiIndex.from_tuples(
        [("Location", "Name"), ("Location", "Longitude"), ("Location", "Latitude")] + 
        [("Date", pd.to_datetime(x, format="%m").month_name()) 
         for x in df_loc_dt.columns[3:]], 
    )
    df_loc_dt.columns = cols
    
else:
    # For this cohort we do not have month data, so show a breakdown of sample
    # numbers by location only.
    
    # Construct a pivot table counting samples.
    df_loc_dt = df_samples.groupby(["location", "longitude", "latitude"]).agg({"sample_id": "count"})

    # Tidy up the columns using a multi index.
    df_loc_dt.reset_index(inplace=True)
    cols = pd.MultiIndex.from_tuples(
        [("Location", "Name"), ("Location", "Longitude"), ("Location", "Latitude"), ("Date", cohort.year)]
    )
    df_loc_dt.columns = cols

# Style the table.
df_loc_dt_styled = (
    df_loc_dt
    .style
    .format(precision=3)
    .set_caption("Number of samples collected.")
    .hide()
)
display(df_loc_dt_styled)    

## Diagnostics
### H12 calibration

In [17]:
display(Markdown(f"Selected window size: **{h12_window_size:,}**"))

window_sizes = (100, 200, 500, 1000, 2000, 5000, 10000, 20000)

ag3.plot_h12_calibration(
    contig=h12_calibration_contig,
    analysis=phasing_analysis,
    sample_sets=sample_sets,
    sample_query=cohort['sample_query'],
    min_cohort_size=min_cohort_size,
    max_cohort_size=max_cohort_size,
    window_sizes=window_sizes,
);


Selected window size: **10,000**

### G123 Calibration

In [18]:
display(Markdown(f"Selected window size: **{g123_window_size:,}**"))

ag3.plot_g123_calibration(
    contig=h12_calibration_contig,
    sites=phasing_analysis,
    site_mask=phasing_analysis,
    sample_sets=sample_sets,
    sample_query=cohort['sample_query'],
    min_cohort_size=min_cohort_size,
    max_cohort_size=max_cohort_size,
    window_sizes=window_sizes,
);

Selected window size: **5,000**

## Data sources

In [19]:
df_sources_style = (
    df_contributors
    .style
    .set_caption("MalariaGEN Vector Observatory partners, studies and sample sets contributing data for this cohort.")
)
df_sources_style

,,,Sample set
Contributor,Study,Data release,
Joao Pinto,Ag1000G,Ag3.0,AG1000G-MZ
